In [1]:
import os
import argparse
import json
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Lambda, ELU
from keras.layers.convolutional import Convolution2D



Using TensorFlow backend.


## Things to try
- New NN architecture

    - 

### Done
- Try Comma AI model
- Try Nvidia Model
- Augment data with left and right images

- Preprocess input
    - Black and White: tried it, did not produce better results

In [2]:
def get_commaai_model(img_shape):
    """
    Creates the comma.ai model, and returns a reference to the model
    The comma.ai model's original source code is available at:
    https://github.com/commaai/research/blob/master/train_steering_model.py
    """
    row, col, ch = img_shape  # camera format

    model = Sequential()
    model.add(Lambda(lambda x: x/127.5 - 1.,
        input_shape=(row, col, ch),
        output_shape=(row, col, ch)))
    model.add(Convolution2D(16, 8, 8, subsample=(4, 4), border_mode='same'))
    model.add(ELU())
    model.add(Convolution2D(32, 5, 5, subsample=(2, 2), border_mode='same'))
    model.add(ELU())
    model.add(Convolution2D(64, 5, 5, subsample=(2, 2), border_mode='same'))
    model.add(Flatten())
    model.add(Dropout(.2))
    model.add(ELU())
    model.add(Dense(512))
    model.add(Dropout(.5))
    model.add(ELU())
    model.add(Dense(1))

    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

    return model

In [3]:
def get_nvidia_model(img_shape):
    """
    Creates the comma.ai model, and returns a reference to the model
    The comma.ai model's original source code is available at:
    https://github.com/commaai/research/blob/master/train_steering_model.py
    """
    row, col, ch = img_shape  # camera format

    model = Sequential()
    model.add(Convolution2D(24, 5, 5, input_shape=(row, col, ch), subsample=(4, 4), border_mode='same'))
    model.add(ELU())
    model.add(Convolution2D(36, 5, 5, subsample=(2, 2), border_mode='same'))
    model.add(ELU())
    model.add(Convolution2D(48, 5, 5, subsample=(2, 2), border_mode='same'))
    model.add(ELU())
    model.add(Convolution2D(64, 3, 3, subsample=(2, 2), border_mode='same'))
    model.add(ELU())
    model.add(Convolution2D(64, 3, 3, subsample=(2, 2), border_mode='same'))
    model.add(Flatten())
    model.add(Dropout(0.2))
    model.add(ELU())
    model.add(Dense(1164))
    model.add(Dropout(0.2))
    model.add(ELU())
    model.add(Dense(100))
    model.add(ELU())
    model.add(Dense(50))
    model.add(ELU())
    model.add(Dense(10))
    model.add(Dense(1))

    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

    return model

In [4]:
import cv2
import numpy as np
import pandas as pd
import skimage.transform as sktransform
import matplotlib.pyplot as plt
import pickle


In [5]:


def crop_image(image, top_offset=.375, bottom_offset=.125):
    """
    Applies preprocessing pipeline to an image: crops `top_offset` and `bottom_offset`
    portions of image, resizes to 32x128 px and scales pixel values to [0, 1].
    """
    top = int(top_offset * image.shape[0])
    bottom = int(bottom_offset * image.shape[0])
    image = sktransform.resize(image[top:-bottom, :], (32, 128, 3))
    return image


def get_data(log_file):
    
    angles = ['center', 'left', 'right']
    angle_offset = [0, 0.25, -0.25]
    
    # Read log file and get root directory
    df = pd.read_csv(log_file, sep=',')
    root_dir = '/'.join(log_file.split('/')[:-1])
    
    features = []
    labels = []
    
    for i, row in df.iterrows():
        for j, camera in enumerate(angles):
            
            # If image does not exist, skip it
            # Checking if flot because Pandas puts float(0) in empty cells
            if not row[j] or type(row[j]) is float:
                continue
            
            img_path = root_dir + '/' + row[j].strip()
            img = cv2.imread(img_path)
            img = crop_image(img)
            
            angle = float(row[3]) + angle_offset[j]
            
            features.append(img)
            labels.append(angle)
    
    features = np.asarray(features)
    labels = np.asarray(labels)
    return features, labels

features, labels = get_data('data/driving_log.csv')
print('Done')


Done


In [6]:
# data = {
#     'features': features,
#     'labels': labels
# }
# with open('data.p', 'w') as f:
#     pickle.dump(data, f)

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(features, labels, test_size=0.3)

In [ ]:
print(X_train.shape[1:])

model = get_commaai_model(X_train.shape[1:])
history = model.fit(X_train, y_train, nb_epoch=5)
print(history.history)

results = model.evaluate(X_val, y_val)
print(results)

model.save('comma_model.h5')

In [ ]:
nvidia_model = get_nvidia_model(X_train.shape[1:])
history = nvidia_model.fit(X_train, y_train, nb_epoch=5)
print(history.history)

results = nvidia_model.evaluate(X_val, y_val)
print(results)
nvidia_model.save('nvidia_model.h5')

Epoch 1/5
20232/20232 [==============================] - 14s - loss: 0.0358 - acc: 0.1548      